In [1]:
%load_ext lab_black

In [2]:
# modules
import numpy as np
import pandas as pd
import seaborn as sns
import pycountry_convert as pc

import re
import spacy
import swifter

from functools import reduce
from tqdm import tqdm
from os import listdir
from os.path import join

In [3]:
# params
datapath = "./../data/data.csv"
out_path = "./../data/data_enriched.pkl"
nlp = spacy.load("en_core_web_sm")

In [4]:
df = pd.read_csv(datapath, encoding="unicode_escape")
df = df.where(pd.notnull(df), None)
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


# EDA Part II - Enriching dataset

In [5]:
# null
df = df[~((df.Description.isna()) & (df.UnitPrice == 0) & (df.CustomerID.isna()))]

# UnitPrice
df.UnitPrice = df.UnitPrice.mask(df.UnitPrice == 0, None)
df = df[(df.UnitPrice > 0) | (df.UnitPrice.isna())]

# InvoiceNo
df.InvoiceNo = df.InvoiceNo.apply(lambda x: str.upper(x) if x is not None else None)
df.InvoiceNo = df.InvoiceNo.apply(lambda x: str.strip(x) if x is not None else None)

# Description
df.Description = df.Description.map(lambda x: str.upper(x) if x is not None else None)
df.Description = df.Description.map(
    lambda x: str.replace(x, ".", "") if x is not None else None
)
df.Description = df.Description.map(lambda x: str.strip(x) if x is not None else None)

# InvoiceDate
df.InvoiceDate = df.InvoiceDate.astype("datetime64")

# CustomerID
df.CustomerID = df.CustomerID.astype("Int64")

# Country
df.Country = df.Country.map(lambda x: None if x == "Unspecified" else x)

In [6]:
df

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART,6,2010-12-01 08:26:00,3.39,17850,United Kingdom
...,...,...,...,...,...,...,...,...
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,2011-12-09 12:50:00,0.85,12680,France
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,2011-12-09 12:50:00,2.10,12680,France
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,2011-12-09 12:50:00,4.15,12680,France
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,2011-12-09 12:50:00,4.15,12680,France


## Combining columns 

In [7]:
df["FullPrice"] = df.Quantity * df.UnitPrice

## Extracting data from non-atomic columns

Extracting number and letter code from `InvoiceNo`.

In [8]:
df["InvoiceCode"] = df.InvoiceNo.apply(lambda x: x[0] if len(x) == 7 else None)
df["InvoiceNumber"] = df.InvoiceNo.apply(lambda x: x if len(x) == 6 else x[1:])

df.drop(columns=["InvoiceNo"], inplace=True)

Extracting number, letter and mark (if `StockCode` is not regular) from `StockCode`.

In [9]:
typical_code_regex = "^([0-9]+)([A-Za-z])?$"

In [10]:
df["Regex"] = df.StockCode.apply(lambda x: re.match(typical_code_regex, x))
df["Regex"].fillna(pd.NA, inplace=True)

df["Regex"] = df.Regex.apply(lambda x: x.groups() if x is not pd.NA else (None, None))

In [11]:
df["StockNumber"] = df.Regex.apply(lambda x: x[0])
df["StockMark"] = df.apply(
    lambda x: x["StockCode"] if x["Regex"] == (None, None) else None, axis=1
)

# overriding column name to stick to name convention
df["StockCode"] = df.Regex.apply(lambda x: x[1])

df.drop(columns=["Regex"], inplace=True)

In [12]:
df

,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,FullPrice,InvoiceCode,InvoiceNumber,StockNumber,StockMark
0,A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.30,None,536365,85123,None
1,None,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34,None,536365,71053,None
2,B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,22.00,None,536365,84406,None
3,G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34,None,536365,84029,None
4,E,RED WOOLLY HOTTIE WHITE HEART,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34,None,536365,84029,None
...,...,...,...,...,...,...,...,...,...,...,...,...
541904,None,PACK OF 20 SPACEBOY NAPKINS,12,2011-12-09 12:50:00,0.85,12680,France,10.20,None,581587,22613,None
541905,None,CHILDREN'S APRON DOLLY GIRL,6,2011-12-09 12:50:00,2.10,12680,France,12.60,None,581587,22899,None
541906,None,CHILDRENS CUTLERY DOLLY GIRL,4,2011-12-09 12:50:00,4.15,12680,France,16.60,None,581587,23254,None
541907,None,CHILDRENS CUTLERY CIRCUS PARADE,4,2011-12-09 12:50:00,4.15,12680,France,16.60,None,581587,23255,None


Extracting `ROOT` part of `Description` in order to retrive minimal info about product.

*Apparently, such an operation requires too much RAM for author of the notebook. There are, of course, solution to make it more* **strem-like** *proccess; however, data obtained in this way is not that important for whole analysis to continue the code listed below:*

In [13]:
def get_subject(doc: spacy.tokens.doc.Doc) -> str or None:
    subjects = [token.lemma_ for token in doc if token.dep_ == "ROOT"]

    return str.upper(subjects[0])

In [14]:
def make_docs(
    col: pd.Series, batch_size: int = 1000, backup_path: str = "../data/docs/"
) -> None:
    times = col.size // batch_size
    last_rows = np.mod(col.size, batch_size)

    def _create_path(i: int) -> str:
        return join(backup_path, "docs_" + str(i) + ".pkl")

    for i in tqdm(range(times)):
        res = pd.DataFrame(
            col[i * batch_size : (i + 1) * batch_size].apply(
                lambda x: nlp(x) if x is not None else None
            )
        )
        res.to_pickle(_create_path(i))

    # last iteration
    res = pd.DataFrame(
        col[times * batch_size : col.size].apply(
            lambda x: nlp(x) if x is not None else None
        )
    )
    res.to_pickle(_create_path(times + 1))

In [15]:
backup_path = "../data/docs/"

# make_docs(df.Description, backup_path='../data/docs/docs')

In [16]:
# docs_files = listdir(backup_path)
# docs_files = sorted(docs_files, key= lambda x: int(re.search('([0-9]+)', x).group()))

In [17]:
# docs = pd.Series()

# for doc in tqdm(docs_files):
#     new_doc = pd.read_pickle(join(backup_path, doc))
#     docs = pd.concat([docs, new_doc])

Extracting continent from `Country`.

In [18]:
countries = df.Country.unique()
continents = []

In [19]:
countries = list(
    filter(
        lambda x: x
        not in ("Channel Islands", None, "EIRE", "European Community", "RSA"),
        countries,
    )
)

In [20]:
for country in countries:
    code = pc.country_name_to_country_alpha2(country)
    continents.append(pc.country_alpha2_to_continent_code(code))

In [21]:
country_continent = {}

for country, continent in zip(countries, continents):
    country_continent[country] = continent

In [22]:
country_continent["Channel Islands"] = "EU"
country_continent["EIRE"] = "EU"
country_continent["European Community"] = "EU"
country_continent["RSA"] = "AF"
country_continent[None] = None

In [23]:
df["Continent"] = df.Country.map(country_continent)

Saving data

In [24]:
new_order = [
    "InvoiceCode",
    "InvoiceNumber",
    "StockNumber",
    "StockCode",
    "StockMark",
    "Description",
    "Quantity",
    "FullPrice",
    "InvoiceDate",
    "UnitPrice",
    "CustomerID",
    "Country",
    "Continent",
]
df = df[new_order]

In [25]:
df.to_pickle(out_path)